### Import Library

In [64]:
import sys
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import c3d
import importlib
from typing import List

# Import helpers
sys.path.append('/app/hpe-core/datasets/vicon_processing/v2')
import helpers

# Import bimvee
sys.path.append('./submodules/bimvee')
from bimvee.importIitYarp import importIitYarp

### Load parameters

In [ ]:
# vicon_c3d_file: str = '/data/test7/VICON/test701.c3d'
# events_folder: str = '/data/test7/left'
# calib_path: str = '/data/test7/calib_left.txt'
# period: float = 1.0 / 100  # vicon frequency: 100HZ
# input_label_tag_file: str = '../scripts/config/labels_tags_vicon.yml'
# output_label_yml_file: str = 'labels_all_test7_tmp_left.yml'
# output_file: str = 'output_tmp_left.yml'

In [65]:
# TODO: make it so that it is not hard-coded

vicon_c3d_file: str = '/home/cappe/hpe/move-iit-hpe-subset1/P1/tennis_s1.c3d'
events_folder: str = '/home/cappe/hpe/move-iit-hpe-subset1/P1/tennis_s1/atis-d/'
calib_path: str = '/home/cappe/hpe/move-iit-hpe-subset1/P1/calib-d.txt'
period: float = 1.0 / 100  # vicon frequency: 100HZ
input_label_tag_file: str = '../scripts/config/labels_tags.yml'
output_label_yml_file: str = 'corrections_labels.yml'
output_file: str = 'output_tmp_left.txt'
subject = 'P1'

### Import event data

In [ ]:
# TODO: modify import of event data so to use "importAe" to fix the memory issue
# importAe will load event data in chunks allowing to laod everytime what is needed

# from bimvee.importAe import importAe

# importers = importAe(events_folder)

In [ ]:
# TODO: correctly laod event data

# print(importers.keys())

# imp = importers['data']['']['dvs']
# imp.get_full_data_as_dict()

# imp.get_data_at_time(4, 0.03)

{'x': array([1121, 1130,  691, ...,  545,  917,  766], dtype=uint16),
 'y': array([287, 287, 421, ..., 207, 207, 206], dtype=uint16),
 'pol': array([ True,  True,  True, ...,  True,  True,  True]),
 'ts': array([ 0.        ,  0.        ,  0.        , ..., 17.08306003,
        17.08306003, 17.08306003])}

In [3]:
# TODO: replace this with importAe, as to load bigger datasets

v_data: dict = importIitYarp(filePathOrName=events_folder, template=None, wrapTime=0.0)
e_ts: np.ndarray = v_data['data']['left']['dvs']['ts']
e_us: np.ndarray = v_data['data']['left']['dvs']['x']
e_vs: np.ndarray = v_data['data']['left']['dvs']['y']

importIitYarp trying path: /home/cappe/hpe/move-iit-hpe-subset1/P1/tennis_s1/atis-d/


100%|██████████| 19091/19091 [00:34<00:00, 560.42it/s]


Examining info.log: /home/cappe/hpe/move-iit-hpe-subset1/P1/tennis_s1/atis-d/info.log


### Import C3D data

In [66]:
c3d_data = c3d.Reader(open(vicon_c3d_file, 'rb'))
points_3d: dict = {}
for i, points, analog in c3d_data.read_frames():
    points_3d[i] = points

marker_t = np.linspace(0.0, c3d_data.frame_count / c3d_data.point_rate, 
                            c3d_data.frame_count, endpoint=False)

/home/cappe/hpe/hpe-core/datasets/vicon_processing/v2/.venv/lib/python3.10/site-packages/c3d/c3d.py:1219: UserWarning: No analog data found in file.
  warnings.warn('No analog data found in file.')


In [67]:
# for hpe i only loaded these markers as otherwise it was too confusing when visualizing
marker_names = ['P1:LANK', 'P1:RANK', 'P1:LKNE', 'P1:RKNE', 'P1:RSHO', 'P1:LSHO', 'P1:LELB', 'P1:RELB', 'P1:LWRA', 'P1:RWRA', 'P1:CLAV', 'P1:STRN', 'P1:LFHD', 'P1:RFHD'] 

# marker names for calibration sequence
# marker_names = ['stereoatis:cam_right          ', 'stereoatis:cam_back           ',
#        'stereoatis:cam_left           ', 'monitor:Origin_mon_top_right  ',
#        'monitor:mon_bottom_right      ', 'monitor:mon_bottom_left       ',
#        'monitor:mon_top_left          ', 'monitor:base_origin_1         ',
#        'monitor:base_2                ', 'monitor:base_3                ',
#        'monitor:base_4                ', 'chair:origin_right_top        ',
#        'chair:right_mid               ', 'chair:right_bottom            ',
#        'chair:left_top                ', 'chair:left_bottom             ',
#        'box2:top_right_Origin         ', 'box2:bottom_right             ',
#        'box2:bottom_left              ', 'box2:top_left                 ',
#        'box1:top_1_origin             ', 'box1:top_2                    ',
#        'box1:top_3                    ', 'box1:top_4                    ',
#        'box1:bottom_1                 ', 'box1:bottom_2                 ',
#        'box1:bottom_3                 ', 'box1:bottom_4                 ']

# uncomment to load all markers by reading them from the c3d file

# marker_names = [f"{name}" for name in c3d_data.point_labels]
# marker_names = [name.strip() for name in marker_names]

# print(marker_names)

### Import calibration data

In [69]:
calib = np.genfromtxt(calib_path, delimiter=" ", skip_header=1, dtype=object)

calib_dict = {}
keys = calib[:, 0].astype(str)
vals = calib[:, 1].astype(float)
calib_dict = {
    key: value for key, value in zip(keys, vals)
}
cam_res = np.int64([calib_dict['h'], calib_dict['w']])
cam_res_rgb = np.append(cam_res, 3)

# intrinsic
K = np.array([
    [calib_dict['fx'], 0.0, calib_dict['cx']],
    [0.0, calib_dict['fy'], calib_dict['cy']],
    [0.0, 0.0, 1.0]
])
D = np.array([calib_dict['k1'], calib_dict['k2'], calib_dict['p1'], calib_dict['p2']])


### Visualize event data as time window

In [191]:
img = np.ones(cam_res, dtype = np.uint8)*255

ft = e_ts[0]
for i in range(0,len(e_ts)):
    #clear the frame at intervals
    if e_ts[i] >= ft:
        cv2.imshow('Visualisation', img)
        k = cv2.waitKey(np.int64(period*1000))
        if k == 27:  # ESC key to stop
            break
        img = np.ones(cam_res, dtype = np.uint8)*255
        text = f"t = {ft:.6f}s"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.7
        thickness = 2
        color = (0, 0, 0)
        (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, thickness)
        x = img.shape[1] - text_width - 10
        y = text_height + 10
        cv2.putText(img, text, (x, y), font, font_scale, color, thickness, cv2.LINE_AA)
        ft = ft + period
    img[e_vs[i],e_us[i]] = 0
    
cv2.destroyAllWindows()

### Visualize marker positions

In [24]:
%matplotlib tk
marker_points = helpers.marker_p(c3d_data.point_labels, points_3d.values(), 'camera:cam_right')
plt.plot(marker_t, marker_points[:, 0:3])
plt.show()

In [ ]:
# delay = 0.365     # calibration
# delay = -0.253    # hpe 

# this will be loaded either from the file or from the user estimations, i left them here for reference for the two sequences i am using the most

In [70]:
# if file exists, load transforamtion and delay, otherwise run the next cells

import os
import numpy as np

file = "/home/cappe/hpe/move-iit-hpe-subset1/P1/tennis_s1/init_file.txt"

if os.path.exists(file):
    with open(file, 'r') as f:
        content = f.read().splitlines()

    # Extract transformation matrix
    start = content.index("[TRANSFORMATION MATRIX SYSTEM TO CAMERA]") + 1
    end = content.index("[DELAY]")  # matrix stops before delay tag
    matrix_lines = content[start:end]

    matrix = []
    for line in matrix_lines:
        # clean brackets [[ ]] and []
        clean = line.replace("[[", "").replace("]]", "").replace("[", "").replace("]", "")
        row = [float(x) for x in clean.strip().split() if x]
        if row:
            matrix.append(row)

    T_w2s_opt = np.array(matrix, dtype=np.float64)

    # Extract delay (skip empty lines after [DELAY])
    delay_index = content.index("[DELAY]") + 1
    while delay_index < len(content) and not content[delay_index].strip():
        delay_index += 1
    delay = float(content[delay_index].strip())
else:
    print("Init file not found, please run the initialization cells.")
    T_w2s_opt = np.eye(4)
    delay = 0.0
    
print("WorldToCamera T:\n", T_w2s_opt)
print("Delay:", delay)


WorldToCamera T:
 [[ 9.08229422e-01  3.81105945e-01  1.72851308e-01 -4.57332783e+02]
 [ 1.54536209e-02 -4.43315943e-01  8.96232202e-01  2.07016366e+02]
 [ 4.18187161e-01 -8.11313277e-01 -4.08522050e-01  3.61449576e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Delay: -0.253


### Extract world to system trasnformation matrices for each timestamp

In [71]:
importlib.reload(helpers)
from helpers import ViconHelper

# Define vicon helper
vicon_helper = ViconHelper(marker_t, points_3d, delay, c3d_data.frame_count, c3d_data.point_rate, c3d_data.point_labels, True, True)

Ts_world_to_system = vicon_helper.compute_camera_marker_transforms()

#Ts_world_to_system

out {'points': [{'camera:cam_right': array([-3240.8223 ,  1006.21356,  1059.4978 ], dtype=float32), 'camera:cam_back': array([-3322.0935,  1026.7214,  1099.2441], dtype=float32), 'camera:cam_left': array([-3268.482 ,  1057.1323,  1060.7485], dtype=float32)}, {'camera:cam_right': array([-3240.8474,  1006.1674,  1059.4875], dtype=float32), 'camera:cam_back': array([-3322.0916,  1026.7482,  1099.2522], dtype=float32), 'camera:cam_left': array([-3268.4846,  1057.1528,  1060.7578], dtype=float32)}, {'camera:cam_right': array([-3240.836 ,  1006.191 ,  1059.4866], dtype=float32), 'camera:cam_back': array([-3322.0933,  1026.7639,  1099.254 ], dtype=float32), 'camera:cam_left': array([-3268.4727,  1057.1958,  1060.7354], dtype=float32)}, {'camera:cam_right': array([-3240.8306,  1006.2223,  1059.4626], dtype=float32), 'camera:cam_back': array([-3322.0723,  1026.7327,  1099.2521], dtype=float32), 'camera:cam_left': array([-3268.5195,  1057.1627,  1060.7761], dtype=float32)}, {'camera:cam_right': 

### Save/Visualize dynamic projection video

In [76]:
importlib.reload(helpers)

projected_points = helpers.project_vicon_to_event_plane_dynamic(
    marker_names, c3d_data, points_3d, marker_t, T_w2s_opt, Ts_world_to_system, K, cam_res, delay, e_ts, e_us, e_vs, period, visualize=True, video_record=False, D=D
)

### Input an estimation of the translation

In [51]:
#TODO: add cell asking user to input the translation

tvec_init = np.array([0.0, 0.0, 0.0])  # in centimeters, x,y,z
# for calibration -> LEFT: x -> between 5.5cm and 11.5cm, y -> -5cm, z -> -0.5cm
#                   RIGHT: x -> between 5.5cm and 11.5cm, y -> 12.7cm, z -> -0.5cm

### Manually estimate rotation

In [81]:
importlib.reload(helpers)
from scipy.spatial.transform import Rotation

#TODO: add cell to manually adjust rotation -> roll, pitch, yaw angles, then save it and combine it with translation from before

R_init = np.array([-119.0, 9.0, -23.0])
R_mat = Rotation.from_euler('zyx', [R_init[2], R_init[1], R_init[0]], degrees=True).as_matrix()

marker_names = ['P1:LANK', 'P1:RANK', 'P1:LKNE', 'P1:RKNE', 'P1:RSHO', 'P1:LSHO', 'P1:LELB', 'P1:RELB', 'P1:LWRA', 'P1:RWRA', 'P1:CLAV', 'P1:STRN', 'P1:LFHD', 'P1:RFHD']
chosen_one = 'P1:CLAV'  # marker to have the coordinate printed on screen so to be able to adjust the rotation if the projections are outside of the window

init_rvec = helpers.project_and_manual_rotation(
    marker_names, c3d_data, points_3d, marker_t, T_w2s_opt, Ts_world_to_system, K, cam_res, delay, e_ts, e_us, e_vs, period, visualize=True, D=D, chosen_one=chosen_one, R_init=R_mat, tvec=tvec_init
)

init_rvec

T_w2s_opt[:3, :3] = cv2.Rodrigues(init_rvec)[0]
T_w2s_opt[:3, 3] = tvec_init

T_w2s_opt

Recomputed projections: roll=-119.50, pitch=9.00, yaw=-23.00
[recalc] marker='P1:CLAV' frame_idx=106 image_uv=(1662.6084124154527, -280.16502649803016)
Recomputed projections: roll=-120.00, pitch=9.00, yaw=-23.00
[recalc] marker='P1:CLAV' frame_idx=143 image_uv=(1610.4382862662076, -209.16785562958052)
Space pressed, visualization paused at markers: 1.640s, events: 1.387s
Recomputed projections: roll=-120.50, pitch=9.00, yaw=-23.00
[recalc] marker='P1:CLAV' frame_idx=164 image_uv=(1540.798254370634, -148.02974252550143)
Recomputed projections: roll=-120.00, pitch=9.00, yaw=-23.00
[recalc] marker='P1:CLAV' frame_idx=164 image_uv=(1536.8247579596627, -159.78070476048248)
Recomputed projections: roll=-119.50, pitch=9.00, yaw=-23.00
[recalc] marker='P1:CLAV' frame_idx=164 image_uv=(1532.9356730189847, -171.60379108836077)


array([[ 0.90917191,  0.38592058,  0.15643447,  0.        ],
       [ 0.06707514, -0.50647774,  0.85964017,  0.        ],
       [ 0.41098341, -0.77106784, -0.48636101,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

### Manually correct delay

In [78]:
importlib.reload(helpers)

delay = helpers.fix_delay(
    marker_names, c3d_data, points_3d, marker_t, T_w2s_opt, Ts_world_to_system, K, cam_res, delay, e_ts, e_us, e_vs, period, visualize=True, D=D
)

delay

Space pressed, visualization paused at markers: 2.060s, events: 1.807s
Space pressed, visualization resumed from markers: 2.060s, events: 1.807s
Space pressed, visualization paused at markers: 2.240s, events: 1.987s
Space pressed, visualization resumed from markers: 2.240s, events: 1.987s
Space pressed, visualization paused at markers: 3.100s, events: 2.847s
Space pressed, visualization resumed from markers: 3.100s, events: 2.847s
Space pressed, visualization paused at markers: 5.650s, events: 5.397s
Space pressed, visualization resumed from markers: 5.650s, events: 5.397s
Space pressed, visualization paused at markers: 5.820s, events: 5.567s
Space pressed, visualization resumed from markers: 5.820s, events: 5.567s
Space pressed, visualization paused at markers: 6.020s, events: 5.767s
Space pressed, visualization resumed from markers: 6.020s, events: 5.767s
Space pressed, visualization paused at markers: 7.110s, events: 6.857s
Space pressed, visualization resumed from markers: 7.110s, 

-0.253

### Save/Visualize dynamic projection video

In [ ]:
importlib.reload(helpers)

projected_points = helpers.project_vicon_to_event_plane_dynamic(
    marker_names, c3d_data, points_3d, marker_t, T_w2s_opt, Ts_world_to_system, K, cam_res, delay, e_ts, e_us, e_vs, period, visualize=False, save_video=True, D=D
)

### Label sequence with list of markers

In [79]:
importlib.reload(helpers)
from helpers import DvsLabeler

labels_path = os.path.join(os.path.dirname(vicon_c3d_file), output_label_yml_file)

time_tags, event_indices = helpers.calc_indices(e_ts, period)

img = np.ones(cam_res, dtype = np.uint8)*255

labeler = DvsLabeler(img_shape=(720, 1280, 3))
labeler.label_data(e_ts, e_us, e_vs, event_indices, time_tags, period, input_label_tag_file)
labeler.save_labeled_points(labels_path)

print("Saved labeled points at:", labels_path)

cv2.destroyAllWindows()

space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
Labeling aborted by user.
ESC pressed, stopping labeling
No points to save. YAML file not created/overwritten: /home/cappe/hpe/move-iit-hpe-subset1/P1/corrections_labels.yml
Saved labeled points at: /home/cappe/hpe/move-iit-hpe-subset1/P1/corrections_labels.yml


### Label sequence with only cv by clicking twice on the window

In [80]:
importlib.reload(helpers)
from helpers import DvsLabeler

# TODO: correct the mistake, as I should have loaded the file with the labels and change them under the correct timestamp instead of starting from scratch

# Define output path for corrections, TODO: change after fix
corrections_path = os.path.join(os.path.dirname(vicon_c3d_file), 'corrections_labels.yml')

corrector = DvsLabeler(img_shape=(720, 1280, 3))
time_tags, event_indices = helpers.calc_indices(e_ts, period)

corrector.correct_data(
    e_ts, e_us, e_vs, event_indices, time_tags, period, 
    marker_names, c3d_data, points_3d, marker_t, T_w2s_opt, Ts_world_to_system, 
    K, cam_res, delay, D
)

# Save corrected results, TODO: fix
corrector.save_labeled_points(corrections_path)

print("Saved corrected points at:", corrections_path)

cv2.destroyAllWindows()

space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, s

AssertionError: 

### Load labels and calculate transformation matrix from system to camera using scipy optimizer

In [38]:
from scipy.optimize import least_squares

def reprojection_error(params, Ps, pc, K, dist):
    """
    params: 6 dimensions (rotation_vector [3], translation [3]) Optimization target
    Ps: (N, 3) 3D points (system coordinates)
    pc: (N, 2) 2D points (image coordinates)
    K: (3, 3) camera matrix
    dist: (5,) distortion parameters (OpenCV format)
    """
    rvec = params[:3]
    tvec = params[3:6]

    # Project 3D system points to 2D image points
    projected_points, _ = cv2.projectPoints(Ps, rvec, tvec, K, dist)
    projected_points = projected_points.reshape(-1, 2)

    return (projected_points - pc).ravel()  # Flatten and return


def estimate_Tstoc(Ps, pc, K, dist, init_params=None):
    """
    Ps: (N, 3) 3D points (system coordinates)
    pc: (N, 2) 2D points (image coordinates)
    K: (3, 3) camera matrix
    dist: (5,) distortion parameters
    return: (4, 4) system→camera coordinate transformation matrix Tstoc
    """

    # Initial values: zero rotation, zero translation
    if init_params is None:
        # If no initial parameters are provided, set them to zero
        init_params = np.zeros(6)

    # Minimize (choose LM method, etc.)
    res = least_squares(
        reprojection_error,
        init_params,
        args=(Ps, pc, K, dist),
        method='lm'  # Levenberg-Marquardt
    )

    rvec_opt = res.x[:3]
    tvec_opt = res.x[3:6]
    R_opt, _ = cv2.Rodrigues(rvec_opt)

    # Construct homogeneous transformation matrix
    T = np.eye(4)
    T[:3, :3] = R_opt
    T[:3, 3] = tvec_opt

    return T

### Label interpolation and generation of 2D-3D correspondences

In [ ]:
importlib.reload(helpers)
from helpers import ViconHelper

# Define vicon helper
vicon_helper = ViconHelper(marker_t, points_3d, delay, c3d_data.frame_count, c3d_data.point_rate, c3d_data.point_labels, True, True)

# Load labeled points
labels_path = os.path.join(os.path.dirname(vicon_c3d_file), output_label_yml_file)
labeled_points = helpers.read_points_labels(labels_path)

# extract the frames_id from the labeled points
frames_id = vicon_helper.get_frame_time(labeled_points['times'])
vicon_points = vicon_helper.get_vicon_points_interpolated(labeled_points)

# Create lists of transformation matrices from world to system
system_points = []
image_points = []
# Ts_world_to_system = vicon_helper.compute_camera_marker_transforms()

# TODO: check out this method thorougly as it may be one of the causes for the errors
# ALSO: this method is not really usefult for events as the event frequency is set to be the same as the vicon frequency

# Collect correspondences from 3D system points to 2D image points
for idx, (dvs_frame, vicon_frame) in enumerate(zip(labeled_points['points'], vicon_points['points'])):
    labels = dvs_frame.keys()

    for l in labels:
        try:
            w_p = vicon_frame[l]
            w_ph = np.append(w_p, 1.0)  # homogeneous coordinates
            i_p = [
                dvs_frame[l]['x'],
                dvs_frame[l]['y']
            ]
            frame_id = vicon_points['frame_ids'][idx]
            p_sys = Ts_world_to_system[frame_id] @ w_ph  # convert to system coordinates
            system_points.append(p_sys[:3])
            image_points.append(i_p)
        except Exception as e:
            print("The stored image labels don't match with the vicon labels used.")
            print(e)
            
system_points = np.array(system_points, dtype=np.float64)
image_points = np.array(image_points, dtype=np.float64)
image_points = image_points[:, :2]

Delay:  -0.253
out {'points': [{'camera:cam_right': array([-3240.8223 ,  1006.21356,  1059.4978 ], dtype=float32), 'camera:cam_back': array([-3322.0935,  1026.7214,  1099.2441], dtype=float32), 'camera:cam_left': array([-3268.482 ,  1057.1323,  1060.7485], dtype=float32)}, {'camera:cam_right': array([-3240.8474,  1006.1674,  1059.4875], dtype=float32), 'camera:cam_back': array([-3322.0916,  1026.7482,  1099.2522], dtype=float32), 'camera:cam_left': array([-3268.4846,  1057.1528,  1060.7578], dtype=float32)}, {'camera:cam_right': array([-3240.836 ,  1006.191 ,  1059.4866], dtype=float32), 'camera:cam_back': array([-3322.0933,  1026.7639,  1099.254 ], dtype=float32), 'camera:cam_left': array([-3268.4727,  1057.1958,  1060.7354], dtype=float32)}, {'camera:cam_right': array([-3240.8306,  1006.2223,  1059.4626], dtype=float32), 'camera:cam_back': array([-3322.0723,  1026.7327,  1099.2521], dtype=float32), 'camera:cam_left': array([-3268.5195,  1057.1627,  1060.7761], dtype=float32)}, {'came

### Static projection, if the camera is static use this method as it uses PnP to produce directly the WorldToCamera T, also if labels are available is a good way to have an initial estimation

##### Note, when working with static camera, results with direct T_world_to_camera trasnsformation matrix might be better, as noise may play a role, I will eventually put a way to differentiate between moving and static camera sequences

In [ ]:
# Collect correspondences and then do projection
world_points = []
image_points_clean = []

for idx, (dvs_frame, vicon_frame) in enumerate(zip(labeled_points['points'], vicon_points['points'])):
    labels = dvs_frame.keys()

    for l in labels:
        try:
            if l not in vicon_frame:
                print(f"Warning: Marker '{l}' not found in Vicon frame {idx}")      # TODO: check as it may be one of the causes for the errors
                continue
                
            w_p = vicon_frame[l]
            
            if w_p is None or np.any(np.isnan(w_p)):
                print(f"Warning: Invalid 3D point for marker '{l}' in frame {idx}")
                continue
                
            i_p = [dvs_frame[l]['x'], dvs_frame[l]['y']]
            
            # Use world coordinates directly
            world_points.append(w_p)
            image_points_clean.append(i_p)
            
        except KeyError as e:
            print(f"KeyError: Marker '{l}' not found in Vicon data at frame {idx}")
        except Exception as e:
            print(f"Error processing marker '{l}': {e}")

world_points = np.array(world_points, dtype=np.float64)
image_points_clean = np.array(image_points_clean, dtype=np.float64)

print(f"Successfully collected {len(world_points)} valid 3D-2D correspondences")
print("World points shape:", world_points.shape)
print("Image points shape:", image_points_clean.shape)

# Solve PnP with world coordinates
success, rvec, tvec = cv2.solvePnP(world_points, image_points_clean, K, D)

if success:
    print("PnP solution successful!")
    
    # Convert to transformation matrix
    R_mat, _ = cv2.Rodrigues(rvec)
    T_world_to_cam = np.eye(4)
    T_world_to_cam[:3, :3] = R_mat
    T_world_to_cam[:3, 3] = tvec.flatten()
    
    print("World-to-camera transformation matrix:")
    print(T_world_to_cam)
    
    # Test projection of a few points, debug
    # if len(world_points) > 0:
    #     test_points = world_points[:min(5, len(world_points))]
    #     projected_test, _ = cv2.projectPoints(test_points, rvec, tvec, K, D)
    #     projected_test = projected_test.reshape(-1, 2)
        
    #     print("Test projections (first few points):")
    #     for i in range(len(projected_test)):
    #         print(f"3D: {test_points[i]} -> 2D projected: {projected_test[i]} (labeled: {image_points_clean[i]})")
            
    projected_points_pnp = helpers.project_vicon_to_event_plane_dynamic(
        marker_names, c3d_data, points_3d, marker_t, T_world_to_cam, 
        {i: np.eye(4) for i in range(len(marker_t))},  # Identity transforms since we're using world-to-camera directly
        K, cam_res, delay, e_ts, e_us, e_vs, period, visualize=True, D=D
    )
    
else:
    print("PnP solution failed!")
    print("Check that you have enough valid correspondences and that the points are not coplanar")

Successfully collected 529 valid 3D-2D correspondences
World points shape: (529, 3)
Image points shape: (529, 2)
PnP solution successful!
World-to-camera transformation matrix:
[[ 4.35883099e-02 -9.95122785e-01  8.84912554e-02  3.95395381e+02]
 [-3.14624878e-02 -8.98988111e-02 -9.95453824e-01  1.11346390e+03]
 [ 9.98554040e-01  4.06059947e-02 -3.52275757e-02  3.67850990e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
# init_T = np.array([[ 5.54118388e-01, -6.46688971e-01, 5.24162298e-01, -1.25350531e+02],
#                    [-3.65516271e-01, -7.54740636e-01, -5.44760888e-01, -8.06780263e+00],
#                    [ 7.47897430e-01, 1.10272191e-01, -6.54591006e-01, -9.41323412e+01],
#                    [ 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

# init_T = np.array([[ 9.58177085e-01,  2.70140431e-01,  9.44500986e-02, -1.10010569e+03],
#                    [ 9.62649522e-02, -6.15063556e-01,  7.82578993e-01, -7.74416582e+02],
#                    [ 2.69499040e-01, -7.40757024e-01, -6.15344861e-01,  4.56237436e+02],
#                    [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

# initial hard-coded estimation for calibration and hpe sequences

In [ ]:
# Create initial guess for the optimization
from scipy.spatial.transform import Rotation

init_T = T_world_to_cam @ np.linalg.inv(Ts_world_to_system[0])      # if labels are available this is a good way to get an initial estimation

r_vec = Rotation.from_matrix(init_T[:3, :3]).as_rotvec()
t_vec = init_T[:3, 3]
r_vec, t_vec, init_T
init_param = np.concatenate((r_vec, t_vec))

init_param

array([-2.03414534e+00, -3.71261191e-01, -4.20229292e-01, -7.05183871e+02,
        6.53372016e+01,  4.20312085e+02])

In [ ]:
T_w2s_opt = estimate_Tstoc(system_points, image_points, K, D, init_param)
T_w2s_opt

array([[ 9.08229422e-01,  3.81105945e-01,  1.72851308e-01,
        -4.57332783e+02],
       [ 1.54536209e-02, -4.43315943e-01,  8.96232202e-01,
         2.07016366e+02],
       [ 4.18187161e-01, -8.11313277e-01, -4.08522050e-01,
         3.61449576e+02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

### Save the transformation matrix from system to camera and the delay

In [47]:
output_file_txt = "/home/cappe/hpe/move-iit-hpe-subset1/P1/tennis_s1/init_file_test.txt"

def _format_matrix_block(T: np.ndarray) -> str:
    rows = []
    for i, row in enumerate(T):
        row_str = " ".join(f"{val: .8e}" for val in row)
        # remove the leading space that comes from the format specifier
        row_str = row_str.lstrip()
        if i == 0:
            rows.append(f"[[ {row_str}]")
        elif i == T.shape[0] - 1:
            rows.append(f" [ {row_str}]]")
        else:
            rows.append(f" [ {row_str}]")
    return "\n".join(rows)

with open(output_file_txt, "w") as f:
    f.write("[TRANSFORMATION MATRIX SYSTEM TO CAMERA]\n\n")
    f.write(_format_matrix_block(T_w2s_opt) + "\n\n")
    f.write("[DELAY]\n\n")
    f.write(f"{delay}\n")
print(f"Saved init_T and delay to {output_file_txt}")

Saved init_T and delay to /home/cappe/hpe/move-iit-hpe-subset1/P1/tennis_s1/init_file_test.txt


### Reproject labels for other sequences using the same transformation matrix from system to camera

In [ ]:
# new_events_path = '/data/test6/left'
# new_vicon_c3d_file: str = '/data/test6/VICON/test6.c3d'
 
# new_v_data: dict = importIitYarp(filePathOrName=new_events_path, template=None, wrapTime=0.0)
# new_e_ts: np.ndarray = new_v_data['data']['left']['dvs']['ts']
# new_e_us: np.ndarray = new_v_data['data']['left']['dvs']['x']
# new_e_vs: np.ndarray = new_v_data['data']['left']['dvs']['y']
 
# new_c3d_data = c3d.Reader(open(new_vicon_c3d_file, 'rb'))
# new_points_3d: dict = {}
# for i, points, analog in new_c3d_data.read_frames():
#     new_points_3d[i] = points
 
# new_marker_t = np.linspace(0.0, new_c3d_data.frame_count / new_c3d_data.point_rate,
#                             new_c3d_data.frame_count, endpoint=False)

new_events_path = '/home/cappe/hpe/move-iit-hpe-subset1/P1/drum_s1/atis-d/'
new_c3d_path = '/home/cappe/hpe/move-iit-hpe-subset1/P1/drum_s1.c3d'

new_ev_data = importIitYarp(filePathOrName=new_events_path, zeroTimestamp=False)
new_e_ts = new_ev_data['data']['left']['dvs']['ts']
new_e_us = new_ev_data['data']['left']['dvs']['x']
new_e_vs = new_ev_data['data']['left']['dvs']['y']

new_c3d_data = c3d.Reader(open(new_c3d_path, 'rb'))
new_points_3d = {}
for i, points, analog in new_c3d_data.read_frames():
    new_points_3d[i] = points

new_marker_t = np.linspace(
    0.0, new_c3d_data.frame_count / new_c3d_data.point_rate,
    new_c3d_data.frame_count, endpoint=False
)

importIitYarp trying path: /home/cappe/hpe/move-iit-hpe-subset1/P1/drum_s1/atis-d/


100%|██████████| 20290/20290 [00:44<00:00, 457.25it/s]


Examining info.log: /home/cappe/hpe/move-iit-hpe-subset1/P1/drum_s1/atis-d/info.log
Video saved as 'tmp.mp4'


In [45]:
importlib.reload(helpers)
from helpers import ViconHelper

# Define vicon helper
new_vicon_helper = ViconHelper(new_marker_t, new_points_3d, delay, new_c3d_data.frame_count, new_c3d_data.point_rate, new_c3d_data.point_labels, True, True)

new_Ts_world_to_system = new_vicon_helper.compute_camera_marker_transforms()

out {'points': [{'camera:cam_right': array([-3226.3208 ,  1008.98566,  1059.2623 ], dtype=float32), 'camera:cam_back': array([-3305.7725,  1035.2411,  1099.3214], dtype=float32), 'camera:cam_left': array([-3253.5872,  1065.7535,  1060.3981], dtype=float32)}, {'camera:cam_right': array([-3226.312 ,  1008.9584,  1059.2499], dtype=float32), 'camera:cam_back': array([-3305.7808,  1035.2263,  1099.3225], dtype=float32), 'camera:cam_left': array([-3253.591 ,  1065.7094,  1060.4044], dtype=float32)}, {'camera:cam_right': array([-3226.325 ,  1008.9697,  1059.2588], dtype=float32), 'camera:cam_back': array([-3305.7598,  1035.217 ,  1099.3146], dtype=float32), 'camera:cam_left': array([-3252.8489,  1062.3002,  1063.1875], dtype=float32)}, {'camera:cam_right': array([-3226.3232 ,  1008.96686,  1059.2596 ], dtype=float32), 'camera:cam_back': array([-3305.7424,  1035.2339,  1099.3085], dtype=float32), 'camera:cam_left': array([-3253.5933,  1065.7748,  1060.3855], dtype=float32)}, {'camera:cam_right

In [ ]:
importlib.reload(helpers)

new_delay = helpers.fix_delay(
    marker_names, new_c3d_data, new_points_3d, new_marker_t, T_w2s_opt, new_Ts_world_to_system, K, cam_res, delay, new_e_ts, new_e_us, new_e_vs, period, visualize=True, video_record=False, D=D
)

# TODO: check camera markers movement -> (big jumps)


In [ ]:
importlib.reload(helpers)

new_delay = -0.245   # add delay fix

new_projected_points = helpers.project_vicon_to_event_plane_dynamic(
    marker_names, new_c3d_data, new_points_3d, new_marker_t, T_w2s_opt, new_Ts_world_to_system, K, cam_res, new_delay, new_e_ts, new_e_us, new_e_vs, period, visualize=True, video_record=False, D=D
)

Delay increased to: -0.243s (step: 0.010s)
Delay step decreased to: 0.009s
Delay step decreased to: 0.008s
Delay step decreased to: 0.007s
Delay step decreased to: 0.006s
Delay step decreased to: 0.005s
Delay step decreased to: 0.004s
Delay step decreased to: 0.003s
Delay step decreased to: 0.002s
Delay step decreased to: 0.001s
Delay decreased to: -0.244s (step: 0.001s)
Delay decreased to: -0.245s (step: 0.001s)
Delay increased to: -0.244s (step: 0.001s)
Delay increased to: -0.243s (step: 0.001s)
Delay increased to: -0.242s (step: 0.001s)
Delay increased to: -0.241s (step: 0.001s)
Delay decreased to: -0.242s (step: 0.001s)
Delay decreased to: -0.243s (step: 0.001s)
Delay decreased to: -0.244s (step: 0.001s)
Delay decreased to: -0.245s (step: 0.001s)
Delay decreased to: -0.246s (step: 0.001s)
Delay decreased to: -0.247s (step: 0.001s)
Delay decreased to: -0.248s (step: 0.001s)
Delay decreased to: -0.249s (step: 0.001s)
Delay increased to: -0.248s (step: 0.001s)
Delay increased to: -0.2